In [ ]:
from splitwise import Splitwise
import pandas as pd
from datetime import datetime, date
from pytz import timezone

#https://github.com/namaggarwal/splitwise
#https://dev.splitwise.com
sw = Splitwise("<consumer key>","<consumer secret>")
current_user = sw.getCurrentUser()
month_start = str(date(2022,7,1))
month_end = str(date(2022,8,1))
print(month_start)
all_expenses = sw.getExpenses(dated_after=month_start, dated_before=month_end)
print(f"Total Expenses Fetched : {len(all_expenses)}")
groups = {group.getId():group for group in sw.getGroups()}

In [22]:
expenses_processed = 0 
excluded_categories = []
custom_category_mapping = {'Music' : 'Subscriptions', 'Utilities - Other' : 'Subscriptions'}
my_expenses = []
for expense in all_expenses:   
    total_expense_cost = expense.getCost()  
    group_id = expense.getGroupId()  
    created_by = expense.getCreatedBy().getFirstName()
    for expense_user in expense.getUsers():        
        if expense_user.getId() == current_user.getId():
            
            category_name = expense.getCategory().getName()
            category_name = custom_category_mapping[category_name] if category_name in custom_category_mapping else category_name
            i_paid = expense_user.getPaidShare()
            i_owe = expense_user.getOwedShare()
            
            # I paid for the whole thing
            # or I added a self-only expense
            # or Excluded categories.
            if 	not (total_expense_cost == i_paid or i_paid == i_owe or i_owe == "0.0" or category_name in excluded_categories):
                    
                my_expense = {'Description' : expense.getDescription(), 
                'Total Amount' : total_expense_cost,
                'Group' : groups[group_id].getName() if group_id in groups else None,
                'Category' : category_name, 
                'Created By' : created_by,
                'I Paid' : i_paid,
                'I Owe' : i_owe,
                'Transaction Date' : datetime.fromisoformat(expense.getDate()[0:19]).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d'),
                'Post Date' : datetime.fromisoformat(expense.getCreatedAt()[0:19]).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d')
                }
                my_expenses.append(my_expense)
                
    expenses_processed += 1


In [ ]:
df = pd.DataFrame(my_expenses)    
df = df.sort_values(by=['Category'])
df.to_csv('data/splitwise-export.csv', index=False)
df